# Title

---
embed-resources: true
echo: false
---

## Introduction

## Methods

In [2]:
# basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint

# model imports
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier

# metric imports
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score

# model selection imports
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# preprocessing imports
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# data imports
from palmerpenguins import load_penguins

In [3]:
# function to calculate the calibration error
def calibration_error(y_true, y_prob, type="expected", n_bins=10):
    """
    Compute calibration error of a binary classifier.

    The calibration error measures the aggregated difference between
    the average predicted probabilities assigned to the positive class,
    and the frequencies of the positive class in the actual outcome.

    Parameters
    ----------
    y_true : array-like of shape (n_samples,)
        True targets of a binary classification task.

    y_prob : array-like of (n_samples,)
        Estimated probabilities for the positive class.

    type : {'expected', 'max'}, default='expected'
        The expected-type is the Expected Calibration Error (ECE), and the
        max-type corresponds to Maximum Calibration Error (MCE).

    n_bins : int, default=10
       The number of bins used when computing the error.

    Returns
    -------
    score : float
        The calibration error.
    """

    bins = np.linspace(0.0, 1.0, n_bins + 1)
    bin_idx = np.searchsorted(bins[1:-1], y_prob)

    bin_sums = np.bincount(bin_idx, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(bin_idx, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(bin_idx, minlength=len(bins))

    nonzero = bin_total != 0
    prob_true = bin_true[nonzero] / bin_total[nonzero]
    prob_pred = bin_sums[nonzero] / bin_total[nonzero]

    if type == "max":
        calibration_error = np.max(np.abs(prob_pred - prob_true))
    elif type == "expected":
        bin_error = np.abs(prob_pred - prob_true) * bin_total[nonzero]
        calibration_error = np.sum(bin_error) / len(y_true)

    return calibration_error

### Data

In [4]:
swing_train = pd.read_parquet(
    "https://lab.cs307.org/swing/data/swing-train.parquet",
)
swing_test = pd.read_parquet(
    "https://lab.cs307.org/swing/data/swing-test.parquet",
)
print (swing_train.head())
print (swing_test.head())

train_mean = swing_train.mean(numeric_only=True)
test_mean = swing_test.mean(numeric_only=True)
swing_train = swing_train.fillna(train_mean)
swing_test = swing_test.fillna(test_mean)

      pitch_name  release_extension  release_pos_x  release_pos_y  \
0         Cutter                6.6          -2.76          53.86   
1       Changeup                6.8          -2.87          53.74   
2       Changeup                6.7          -2.83          53.82   
3  Knuckle Curve                6.7          -2.70          53.78   
4         Cutter                6.7          -2.64          53.83   

   release_pos_z  release_speed  release_spin_rate  spin_axis  plate_x  \
0           5.81           92.6             2376.0      195.0    -0.09   
1           5.66           86.3             1511.0      226.0    -1.47   
2           5.68           87.9             1570.0      224.0    -1.52   
3           5.78           82.4             2398.0       32.0     0.20   
4           5.81           91.0             2427.0      189.0     0.89   

   plate_z  ...  balls  strikes  on_3b  on_2b  on_1b  outs_when_up  stand  \
0     2.79  ...      3        1      0      0      0           

In [5]:
# create X and y for train
X_train = swing_train.drop("swing", axis=1)
y_train = swing_train["swing"]

# create X and y for test
X_test = swing_test.drop("swing", axis=1)
y_test = swing_test["swing"]


In [6]:
# echo: true
# summary statistics

n_samples, n_features = X_train.shape
print(n_samples)
print(n_features)

all = y_train.mean()
print(all)


2663
21
0.47728126173488544


In [7]:
# pitch type
pitch = swing_train.groupby('pitch_name')['swing'].mean().sort_index()
print(pitch)

pitch_name
4-Seam Fastball    0.470904
Changeup           0.536313
Cutter             0.451477
Knuckle Curve      0.462875
Slider             0.496124
Name: swing, dtype: float64


### Models

In [8]:

categorical = ["pitch_name", "balls", "strikes"]
#numerical = X_train.select_dtypes(include=['int64', 'float64']).columns


numerical = ["release_speed", "release_spin_rate", "spin_axis", "plate_x", "plate_z"]


numeric_transformer = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
    ]
)

# define preprocessing for categorical features
categorical_transformer = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder()),
    ]
)

# create general preprocessor
preprocessor = ColumnTransformer(
    [
        ("numeric", numeric_transformer, numerical),
        ("categorical", categorical_transformer, categorical),
    ],
    remainder="drop",
)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier

In [10]:
from sklearn.calibration import CalibratedClassifierCV

In [43]:
#pipeline = Pipeline([
 #   ("preprocessor", preprocessor),
  #  ("classifier", CalibratedClassifierCV(
   #     estimator=RandomForestClassifier(
    #        n_estimators=25,
     #       random_state=42
      #  )
    #))
#])

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", CalibratedClassifierCV(
        estimator=RandomForestClassifier(
            n_estimators=25,
            random_state=42
        ),
        method="isotonic",
        cv=10
    ))
])

In [47]:
#param_grid = {
#    "classifier__estimator__max_features" : [1, 5, 10],
#    "classifier__estimator__max_depth": [5, 10, 15, 25, None],
#    "classifier__estimator__criterion": ["log_loss", "gini"],
#    "classifier__method": [
#        "isotonic",
#        "sigmoid",
#    ],
#}
param_grid = {
    "classifier__estimator__max_features": [1,5,10],
    "classifier__estimator__max_depth": [5, 10, 15,20,25],
   # "classifier__estimator__min_samples_leaf": [5, 10],
    "classifier__estimator__criterion": ["log_loss", "gini"],
    "classifier__method": ["isotonic", "sigmoid"],
}

In [48]:
model = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring="neg_brier_score", #change
)

model.fit(X_train, y_train)

,estimator,Pipeline(step...'isotonic'))])
,param_grid,"{'classifier__estimator__criterion': ['log_loss', 'gini'], 'classifier__estimator__max_depth': [5, 10, ...], 'classifier__estimator__max_features': [1, 5, ...], 'classifier__method': ['isotonic', 'sigmoid']}"
,scoring,'neg_brier_score'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('numeric', ...), ('categorical', ...)]"


In [49]:
model.predict(X_train)
model.predict_proba(X_train)


y_test_pred = model.predict(X_test)
y_test_prob = model.predict_proba(X_test)[:, 1] #change

In [50]:
accuracy_score(y_test, y_test_pred)

0.75

In [51]:
from sklearn.metrics import brier_score_loss

In [52]:
test_brier = brier_score_loss(y_test, y_test_prob)
test_ece = calibration_error(y_test, y_test_prob, type="expected")
test_mce = calibration_error(y_test, y_test_prob, type="max", n_bins = 5)

print(f"test brier score : {test_brier}")
print(f"test_ece : {test_ece}")
print(f"test_mce : {test_mce}")

test brier score : 0.1691500395481634
test_ece : 0.04935004679351834
test_mce : 0.0828804616174606


In [53]:
print(test_brier < 0.19)
print(test_ece < .065)
print(test_mce < .12)

True
True
True


## Results

In [54]:
# report model metrics

In [55]:
# summary figure

In [56]:
# serialize model
import joblib
joblib.dump(model, "swing.joblib", compress=9)


['swing.joblib']

## Discussion